# Neutral Pions at the SBN Program

#### There is no shortage of reasons to study neutral pions at the SBN Program
* Neutral-current (NC) $\pi^{0}$s are primary background in electron neutrino search
* Both charged-current (CC) and NC $\pi^{0}$s offer…
  * Opportunities to probe resonant interaction relevant to all accelerator-based neutrino experiments
  * Standard candles for shower energy calibration given diphoton invariant mass

#### Today we will focus on the following interactions involving BNB neutrinos:
$\nu_{\mu} + Ar \rightarrow 1\mu^{-} + 0\pi^{\pm} +  1\pi^{0} + X$,

where $X$ is any final state particle other than muons or charged pions.  Charged pions are deliberately excluded as to focus on resonant production modes.


In [7]:
# basic imports
import sys
import yaml
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uproot
import ROOT

## SPINE setup

In [8]:
# If you haven't installed spine-ml with pip, point to a local build
SOFTWARE_DIR = '/sdf/data/neutrino/software/spine/src/'
sys.path.append(SOFTWARE_DIR)
from spine.driver import Driver
from spine.vis.out import Drawer

## Configuration
#### Choose experiment (SBND or ICARUS) and initialize SPINE driver

In [9]:
exp = 'sbnd'
#exp = 'icarus'

In [14]:
# Specify path to SPINE HDF5 file
# Note that these files are not guaranteed to be the latest and greatest
if exp == 'sbnd':
    HDF5_PATH = 'data/workshop2025/larcv_mc_20250729_220114_139358-e726b149-4070-45ff-9490-bda42039ffe2_spine.h5'
    #HDF5_PATH = '/sdf/data/neutrino/sbnd/spine/prod/bnb_rockbox_cosmics_v01/output_spine_20250305/larcv_bnb_cosmics_0_spine.h5'
    ENTRY = 10#62
    TRUTH_INTER_ID = 0
elif exp == 'icarus':  
    HDF5_PATH = '/sdf/data/neutrino/icarus/spine/prod/bnb_nu_corsika_sys_250625/cv_respun/output_spine/larcv_mc_20250209_213718_558989-0c639ab3-6c2c-4b9e-9c5b-df908c11dc35_spine.h5'
    # If you have access to SBN GPVMs:
    #HDF5_PATH = '/exp/icarus/data/users/lkashur/spine_workshop_2025/larcv_mc_20250209_213718_558989-0c639ab3-6c2c-4b9e-9c5b-df908c11dc35_spine.h5'
    ENTRY = 2
    TRUTH_INTER_ID = 31

MEDULLA_OUTPUT_PATH = 'data/workshop2025/output/medulla_output_pi0_tutorial.root'
# If you have access to the SBN GPVMs:
#MEDULLA_OUTPUT_PATH = '/exp/icarus/data/users/lkashur/spine_workshop_2025/icarus/icarus_bnb_ccpi0_cv_29_09_2025.root'

In [15]:
# configure
cfg = '''
base:
  iterations: -1
  overwrite_log: true

# Load HDF5 files
io:
  reader:
    file_keys: HDF5_PATH
    name: hdf5
    skip_unknown_attrs: true

# Build reconstruction output representations
build:
  mode: both
  fragments: false
  particles: true
  interactions: true
  units: cm
'''.replace('HDF5_PATH', HDF5_PATH)
cfg = yaml.safe_load(cfg)

In [16]:
driver = Driver(cfg)


 ██████████   ██████████    ███   ███       ██   ███████████
███        █  ██       ███   █    █████     ██   ██         
  ████████    ██       ███  ███   ██  ████  ██   ██████████ 
█        ███  ██████████     █    ██     █████   ██         
 ██████████   ██            ███   ██       ███   ███████████

Release version: 0.7.6

$CUDA_VISIBLE_DEVICES=None

Configuration processed at: Darwin pn2405730.lanl.gov 24.6.0 Darwin Kernel Version 24.6.0: Mon Aug 11 21:16:31 PDT 2025; root:xnu-11417.140.69.701.11~1/RELEASE_ARM64_T6030 arm64

base: {iterations: -1, overwrite_log: true, world_size: 0, seed: 1759872490}
io:
  reader: {file_keys: data/workshop2025/larcv_mc_20250729_220114_139358-e726b149-4070-45ff-9490-bda42039ffe2_spine.h5,
    name: hdf5, skip_unknown_attrs: true}
build: {mode: both, fragments: false, particles: true, interactions: true, units: cm}

Will load 1 file(s):
 - data/workshop2025/larcv_mc_20250729_220114_139358-e726b149-4070-45ff-9490-bda42039ffe2_spine.h5

Total number

In [78]:
ENTRY = 0
data = driver.process(entry=ENTRY)

## Signal Definition

We'll define a signal $\nu_{\mu}$ CC $\pi^{0}$ interaction as any fiducial neutrino interaction resulting in the following final state particles:
* 1 primary muon (KE $\ge$ 143 MeV)
* 0 primary charged pions (KE $\ge$ 25 MeV)
* 1 primary neutral pion.

Using the SPINE drawing tool, let's take a look at a signal interaction in truth:

In [79]:
drawer = Drawer(data, draw_mode='truth', detector=exp) 
fig = drawer.get('particles', ['interaction_id', 'id', 'pdg_code', 'parent_track_id', 'parent_pdg_code'], draw_end_points=False, draw_vertices=False)
fig.show()

## Taking a closer look at the truth information available to us
#### What interaction/particle attributes tell us we have a signal interaction?

In [80]:
# Select truth_interactions key from data dictionary
truth_interactions = data['truth_interactions']

In [81]:
# Pick a specific interaction
truth_inter = [inter for inter in truth_interactions if inter.id == TRUTH_INTER_ID][0]

print(f'Now viewing Truth Interaction {TRUTH_INTER_ID}, containing the following particles:\n')
for p in truth_inter.particles:
    print(p)

Now viewing Truth Interaction 0, containing the following particles:

TruthParticle(ID: 0   | PID: Muon     | Primary: 1  | Size: 91    | Match: 23 )
TruthParticle(ID: 1   | PID: Proton   | Primary: 1  | Size: 6     | Match: 22 )
TruthParticle(ID: 2   | PID: Proton   | Primary: 1  | Size: 1     | Match: -1 )
TruthParticle(ID: 3   | PID: Proton   | Primary: 1  | Size: 15    | Match: 24 )


#### Check for true primary muons

In [69]:
truth_muons = [p for p in truth_inter.particles if p.is_primary and p.pdg_code == 13]
print(f'{len(truth_muons)} true primary muons present\n')
if(len(truth_muons)):
    for p in truth_muons:
        print(f'True Muon ID: {p.id}\nTrue Muon Kinetic Energy: {p.ke:.2f} MeV')
        print('-' * 50)

1 true primary muons present

True Muon ID: 0
True Muon Kinetic Energy: 245.13 MeV
--------------------------------------------------


#### Check for (absence of) true primary charged pions

In [70]:
truth_pions = [p for p in truth_inter.particles if p.is_primary and abs(p.pdg_code) == 211]
print(f'{len(truth_pions)} true primary charged pions present\n')
if(len(truth_pions)):
    for p in truth_pions:
        print(f'True Pion ID: {p.id}, True Pion Kinetic Energy: {p.ke:.2f} MeV')

1 true primary charged pions present

True Pion ID: 1, True Pion Kinetic Energy: 74.28 MeV


#### Check for true primary neutral pions

In [71]:
def find_true_pi0s(truth_inter, primary=True):
    '''
    Find true neutral pions in a true interaction.

    Parameters
    ----------
    truth_inter : TruthInteraction
        The SPINE TruthInteraction object to check.
    primary : bool
        True to check primary neutral pions, False to check nonprimary neutral pions.

    Returns
    -------
    defaultdict(list)
        Dictionary with keys corresponding to neutral pion track ID and values corresponding to neutral pion daughter IDs.
    '''
    truth_pi0s = defaultdict(list)
    for p in truth_inter.particles:
        if primary and not p.is_primary : continue
        if (p.pdg_code == 22 or abs(p.pdg_code) == 11) and p.parent_pdg_code == 111:
            truth_pi0s[p.parent_track_id].append(p.id)
    return truth_pi0s

In [72]:
truth_pi0s = find_true_pi0s(truth_inter)
if len(truth_pi0s):
    print(f'{len(truth_pi0s)} true primary neutral pion present\n')
    for k,v in truth_pi0s.items():
        print(f'True Neutral Pion ID: {k}\nTrue Neutral Pion Daughter IDs: {v}')
        print('-' * 50)
else:
    print('No true neutral pion is present')

truth_pi0_daughters = [p for p in truth_inter.particles if p.id in list(truth_pi0s.values())[0]]

No true neutral pion is present


IndexError: list index out of range

In [ ]:
# Sanity check: 
# Do we have a reasonable handle on the true momentum of the neutral pion?
# This quanity is important in differential cross section measurements
# We will check this using the four-momenta of the pi0 decay products

# P_pi0 = P_sh1 + P_sh2
P_pi0 = np.array([0., 0., 0., 0.])
for sh in truth_pi0_daughters:
    P_pi0 += np.array([sh.energy_init, sh.momentum[0], sh.momentum[1], sh.momentum[2]])

# M_pi0 ^2 = P_pi0 ^2
M_pi0_sqr = P_pi0[0]**2 - np.dot(P_pi0[1:], P_pi0[1:])
M_pi0 = np.sqrt(M_pi0_sqr)
print(f'True neutral pion mass: {M_pi0:.4f} MeV/c^2')

## Applying selection cuts
#### To select $\nu_{\mu}$ CC $\pi^{0}$ candidates, the following selection cuts are applied to reconstructed interactions:
* Interaction vertex inside fiducial volume
* Valid optical flash
* 1 primary muon (KE $\ge$ 143 MeV)
* 0 primary charged pions (KE $\ge$ 25 MeV)
* 2 primary photons (KE $\ge$ 25 MeV)

If you want to maximize your selection purity, cuts on reconstructed neutral pion mass are beneficial (i.e. 50 MeV $\le$ m$_{\gamma \gamma}$ < 400 MeV)

In [ ]:
# Select reco_interactions key from data dictionary
reco_interactions = data['reco_interactions']

In [ ]:
# Loop over interactions and apply selection cuts
selected_interactions = []
for inter in reco_interactions:
    
    # Is interaction fiducial?
    if not inter.is_fiducial: continue

    # Is interaction matched to a valid optical flash?
    if not inter.is_flash_matched: continue
        
    # Does interaction contain one primary muon?
    prim_muons = [p for p in inter.particles if p.is_primary and p.pid == 2 and p.ke > 143]
    if len(prim_muons) != 1: continue
    
    # Does interaction contain zero charged pions?
    prim_pions = [p for p in inter.particles if p.is_primary and p.pid == 3 and p.ke > 25]
    if len(prim_pions) != 0: continue
    
    # Does interaction contain two primary photons?
    prim_photons = [p for p in inter.particles if p.is_primary and p.pid == 0 and p.ke > 25]
    if len(prim_photons) != 2: continue

    # Is invariant mass less than 400 MeV/c^2?
    ph1, ph2 = prim_photons[0], prim_photons[1]
    ph1_energy, ph2_energy = ph1.calo_ke, ph2.calo_ke
    ph1_dir, ph2_dir = ph1.start_point - inter.vertex, ph2.start_point - inter.vertex
    ph1_dir /= np.linalg.norm(ph1_dir)
    ph2_dir /= np.linalg.norm(ph2_dir)
    photons_costheta = np.dot(ph1_dir, ph2_dir)
    pi0_mass = np.sqrt(2*ph1_energy*ph2_energy * (1-photons_costheta))
    if pi0_mass > 400: continue

    # Store output for interactions passing all cuts
    out = {}
    out['reco_interaction_id'] = inter.id
    out['reco_vertex_x'] = inter.vertex[0]
    out['reco_vertex_y'] = inter.vertex[1]
    out['reco_vertex_z'] = inter.vertex[2]
    out['reco_muon_id'] = prim_muons[0].id
    out['reco_ph1_id'] = ph1.id
    out['reco_ph1_calo_ke'] = ph1_energy
    out['reco_ph2_id'] = ph2.id
    out['reco_ph2_calo_ke'] = ph2_energy
    out['reco_photons_costheta'] = photons_costheta
    out['reco_pi0_mass'] = pi0_mass
    out['true_interaction_id'] = inter.match_ids[0]
    
    selected_interactions.append(out)

# List of dictionaries --> pandas dataframe
sel_df = pd.DataFrame(selected_interactions)

In [ ]:
# Each row in our dataframe corresponds to an interaction passing the selection
sel_df

In [ ]:
sel_df.iloc[0]

#### We'll take another look at this event, now in reconstructed space

In [ ]:
drawer = Drawer(data, draw_mode='reco', detector=exp) 
fig = drawer.get('particles', ['interaction_id', 'id', 'pid', 'ke'], draw_end_points=False, draw_vertices=False)
fig.show()

#### A Word on shower energy reconstruction
The particle kinetic energy attribute (`p.ke`) defaults to the following calorimetric sum for photons and electrons:

$$E_{shower} = W_{i} [\frac{MeV}{e^{-}}] \cdot C_{cal} [\frac{e^{-}}{ADC}] \cdot C_{adj} \cdot \frac{1}{R} \cdot \sum_{dep} e^{\frac{t_{drift}}{\tau}} \cdot dep [ADC]$$

with calibration factors:
- $W_{i}$   : Ar work function (23.6 eV/e$^{-}$)
- $C_{cal}$ : Calorimetric gain (ADC $\rightarrow$ electron conversion)
- $C_{adj}$ : Factor to account for subthreshold charge
- $R$       : Recombination factor
- $\tau$    : Electron lifetime

The expression for $E_{shower}$ can be further tweaked a containment correction and neutral pion mass adjustment.

## Medulla
#### Large-scale selection is carried out with CAF-based Medulla framework, which offers support for...
* Sample scaling
* Systematic assessment
* Unblinding decisions

We won't run medulla here, but can take a look at an output ROOT file from a selction in MC here:

In [ ]:
with uproot.open(MEDULLA_OUTPUT_PATH) as rf:

    print('Medulla output file contents:')
    for k in rf.keys(cycle=False):
        print(f'-{k}')

    # TTree --> pandas DataFrame
    medulla_sel_nu_tree = rf['events/cv/selected_nu']
    medulla_sel_cos_tree = rf['events/cv/selected_cos']
    medulla_sig_tree = rf['events/cv/signal']
    medulla_sel_nu_df = medulla_sel_nu_tree.arrays(library='pd')
    medulla_sel_cos_df = medulla_sel_cos_tree.arrays(library='pd')
    medulla_sig_df = medulla_sig_tree.arrays(library='pd')

    expsr_tree = rf['events/cv/selected_nu_exposure']
    expsr_df = expsr_tree.arrays(library='pd')

#### Let's look at our *selected_nu* and *selected_cos* trees

These trees contain all interactions passing our selection cuts that are matched to a true neutrino or true cosmic, respectively.
Once again, each row of the dataframe corresponds to a selected $\nu_{\mu}$ CC $\pi^{0}$ candidate

In [ ]:
# Selected neutrino interactions in MC
medulla_sel_nu_df

In [ ]:
# Selected cosmic interactions in MC
medulla_sel_cos_df

In [ ]:
# Combine nu and cos datasets
medulla_sel_df = pd.concat([medulla_sel_nu_df, medulla_sel_cos_df])

In [ ]:
medulla_sel_df

## Plotting
#### Pick a variable and take a look

In [ ]:
_var = 'reco_pi0_momentum'
_binning = np.linspace(0, 1.2, 13)
xlabel = 'Reco $\pi^{0}$ Momentum [GeV/c]'
ylabel = 'Candidates'

#_var = 'reco_pi0_mass'
#_binning = np.linspace(0, 400, 41)
#xlabel = 'Reco $\pi^{0}$ Momentum [GeV/c]'
#ylabel = 'Candidates'

fig,ax=plt.subplots(figsize=(6,5))
ax.hist(medulla_sel_df[_var], _binning, histtype='step')
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
plt.show()

#### More advanced is plotting is available with medulla's *spineplot*
Normalized MC (left), Normalized MC and data (middle), Including systematics and scaling to data POT (right)
<img src="reco_pi0_momentum_plots.png" alt="reco_pi0_momentum_plots" width="1200"/>

#### Path to physics
The $\nu_{\mu}$ CC $\pi^{0}$ analysis is targeting a series of differential cross section measurements.

Here we'll discuss how to prepare SPINE inputs for the chosen cross section extraction tool of ICARUS: **GUNDAM**

#### What inputs does GUNDAM need?

GUNDAM essentially takes *sbruce* trees as input, with the basic list of ingredients being...
1. Selection tree for data
2. Selection tree for MC
3. Signal tree for MC

\+ relevant POT/livetime scaling factors

These items are direct outputs from medulla, which makes life easy.

Some modifications are required however, and are listed here:
* Categorization variables that define signal and backgrounds must be cast as integers
* Multisigma sigmas and weights must be stored in the MC selected tree as a ROOT TClonesArray for every interaction

#### What does GUNDAM do with these inputs?

The measured cross section in true bin $i$ of a variable $X$ is calculated as:
\begin{equation}
    \frac{\text{d}\sigma}{\text{d}X_{i}} = \sum_{j} \frac{U_{ij} (N_{j} - B_{j})}{\Phi \ \epsilon_{i} \ N_{target}} \frac{1}{\Delta X_{i}},
\end{equation}

where
* $j$ is reconstructed bin
* $N_{j}$ is number of selected events in bin $j$
* $B_{j}$ is number of background events in bin $j$
* $\epsilon_{j}$ is selection efficiency in bin $j$
* $X_{i}$ is the true bin width
* $U_{ij}$ is an unfolding matrix
* $\Phi$ is the integrated neutrino flux
* $N_{target}$ is the number of nuclear targets.

**How GUNDAM uses selection trees:** 
As a binned maxiumum likilihood fitter, GUNDAM fits MC bin counts to data bin counts in reconstructed space through $\chi^{2}$ minimization.  The fit considers free template paramters which control true signal counts and nuisance parameters that describe the flux, interaction, and detector models.  The output of GUNDAM's fitter is the numerator of the differential cross section expression, or the number of background subtracted signal events in true bin $i$

**How GUNDAM uses signal tree:**
Post-fit parameters are propagated to the signal counts, and each analysis bin is subject to efficiency, flux, and number of target corrections.

#### An example cross section
<img src="xsec_pi0_momentum.png" alt="xsec_pi0_momentum" width="400"/>